In [2]:
import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve

In [12]:
df = pd.read_parquet('NF-UNSW-NB15-V2.parquet')

In [14]:
df.columns

Index(['L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES',
       'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS', 'TCP_FLAGS', 'CLIENT_TCP_FLAGS',
       'SERVER_TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'DURATION_IN',
       'DURATION_OUT', 'MIN_TTL', 'MAX_TTL', 'LONGEST_FLOW_PKT',
       'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN',
       'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack'],
      dtype='object')

In [3]:
df.columns

Index(['L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES',
       'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS', 'TCP_FLAGS', 'CLIENT_TCP_FLAGS',
       'SERVER_TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'DURATION_IN',
       'DURATION_OUT', 'MIN_TTL', 'MAX_TTL', 'LONGEST_FLOW_PKT',
       'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN',
       'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack'],
      dtype='object')

In [4]:
features_to_remove = ['L4_SRC_PORT', 'L4_DST_PORT', 'Attack', 'Label']

In [5]:
Y = df.Label

In [6]:
df

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,1305,21,6,1.0,9,1,193,3,24,24,...,0,7240,0,0,0,0,0,331.0,0,Benign
1,1305,21,6,1.0,261,5,469,7,24,24,...,8688,8688,18944,74,0,0,0,230.0,0,Benign
2,1305,21,6,1.0,481,9,750,11,24,24,...,10136,10136,33792,132,0,0,0,229.0,0,Benign
3,1305,21,6,1.0,701,13,1054,15,24,24,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
4,1305,21,6,1.0,1031,19,1474,21,24,24,...,14480,13032,64256,251,0,0,0,230.0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986740,20890,5190,6,0.0,1064,12,2260,12,27,27,...,10136,10136,29696,116,0,0,0,0.0,0,Benign
1986741,58663,5190,6,0.0,1064,12,2364,14,27,27,...,10136,10136,29696,116,0,0,0,0.0,0,Benign
1986742,54553,80,6,7.0,994,10,8896,10,26,26,...,10136,7240,7424,29,0,0,0,0.0,0,Benign
1986743,55026,8248,6,0.0,4014,68,60268,70,27,27,...,44888,14480,8960,35,0,0,0,0.0,0,Benign


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3,stratify=df.Attack, random_state=42)

In [8]:
del df

In [9]:
X_train.drop(columns=features_to_remove, axis=1, inplace=True)
X_test.drop(columns=features_to_remove, axis=1, inplace=True)

KeyError: "['IPV4_SRC_ADDR', 'IPV4_DST_ADDR'] not found in axis"

In [10]:
X_train

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
537464,31891,21,6,1.0,701,13,1052,15,24,24,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
1898859,55420,55517,6,0.0,2230,34,14816,36,27,27,...,20272,14480,6912,27,0,0,0,0.0,0,Benign
1958174,2884,33720,6,0.0,4238,72,63618,74,27,27,...,47784,14480,11008,43,0,0,0,0.0,0,Benign
268989,53080,6881,6,0.0,7532,126,269322,222,27,27,...,65160,21720,8704,34,0,0,0,0.0,0,Benign
908337,19107,14449,17,0.0,512,4,304,4,0,0,...,0,0,0,0,0,0,0,0.0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275414,2450,80,6,7.0,1684,14,10168,18,27,27,...,11584,8688,16896,66,0,0,0,0.0,0,Benign
1012887,31170,56767,6,0.0,2230,34,14816,36,27,27,...,20272,14480,6912,27,0,0,0,0.0,0,Benign
1046591,1932,80,6,7.0,1684,14,10168,18,27,27,...,11584,8688,16896,66,0,0,0,0.0,0,Benign
560859,26610,6881,6,0.0,1540,16,1644,18,27,27,...,10136,10136,64000,250,0,0,0,0.0,0,Benign


In [23]:
X_test

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
719977,6,0.0,8928,14,424,8,27,27,19,0,...,0,6,5792,10136,39936,156,0,0,0,0.0
499337,6,0.0,1036,12,2262,12,27,27,27,0,...,2,0,10136,10136,26112,102,0,0,0,0.0
693932,17,0.0,568,4,304,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
468655,6,0.0,3182,52,38766,54,27,27,27,0,...,0,28,33304,14480,8960,35,0,0,0,0.0
1975227,6,0.0,16432,288,707082,560,27,27,27,0,...,42,454,65160,41992,59392,232,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832216,6,3.0,37228,52,3380,42,27,27,27,0,...,0,24,15928,30408,56832,222,0,0,0,0.0
1401125,6,0.0,8928,14,320,6,27,27,19,0,...,0,6,5792,10136,39936,156,0,0,0,0.0
54146,6,1.0,701,13,1054,15,24,24,24,0,...,0,0,11584,11584,48640,190,0,0,0,125.0
88371,6,0.0,2854,46,30744,48,27,27,27,0,...,2,20,28960,14480,6912,27,0,0,0,0.0


In [24]:
scaler = MinMaxScaler()

In [26]:
X_train = scaler.fit_transform(X_train)

In [27]:
X_test = scaler.transform(X_test)

In [28]:
X_train

array([[2.35294118e-02, 4.08163265e-03, 2.31470696e-05, ...,
        0.00000000e+00, 0.00000000e+00, 2.49500998e-01],
       [2.35294118e-02, 0.00000000e+00, 7.37068832e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.35294118e-02, 0.00000000e+00, 1.40105906e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.35294118e-02, 2.85714286e-02, 5.56521689e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.35294118e-02, 0.00000000e+00, 5.08904860e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.66666667e-02, 0.00000000e+00, 1.71618988e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [29]:
X_test

array([[2.35294118e-02, 0.00000000e+00, 2.95191272e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.35294118e-02, 0.00000000e+00, 3.42245958e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.66666667e-02, 0.00000000e+00, 1.87491264e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.35294118e-02, 4.08163265e-03, 2.31470696e-05, ...,
        0.00000000e+00, 0.00000000e+00, 2.49500998e-01],
       [2.35294118e-02, 0.00000000e+00, 9.43408424e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.35294118e-02, 0.00000000e+00, 1.25820857e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [30]:
clf = CatBoostClassifier(random_seed=42, depth=4, custom_metric=['AUC'], iterations=1000, learning_rate=0.2, subsample=0.95, bootstrap_type='Bernoulli')
clf.fit(X_train, y_train, verbose = False, early_stopping_rounds=100) 

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

NameError: name 'accuracy_score' is not defined